Project SHCT

Authors: Curumi Klejsi, Querzè Lorenzo  
Spring semester 2025

In [1]:
#import os
#os.chdir("/content/SHCT")
#import sys
#sys.path.append("/content/SHCT")
from pylab import *
import numpy as np
import Fluid_CP as FCP #calculation of  thermodynamic states
from scipy.optimize import minimize, NonlinearConstraint
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# PARAMETERS: 

working_time = 43200 # s, 12 hours in seconds

# Pasta

T_pasta_in = 7 # °C, initial pasta temperature 
T_pasta_fin = 85 # °C, final pasta temperature
m_pasta = 2 # kg, total kg of pasta
m_pasta_basket = 0.5 # kg of pasta per basket
t_break = 300 # s, break time in between batches
t_cook = 420 # s, cooking time for the pasta
R_h2o_pasta = 15 # K/kW, thermal resistance between pasta and water
cp_pasta = 3.5 # kJ/kg K, heat capacity of pasta

# H2O (sink)

T_sink_in = 20 # °C, initial water temperature in the morning
T_min_h2o = 85 # °C, minimum water temperature during operation
T_max_h2o = 93 # °C, maximum water temperature during operation
R_h2o_env = 30 # K/kW, thermal resistance between environment and water
cp_h2o = 4.18 # kJ/kg K, heat capacity of water

# Air (source)

T_source = 20 # °C, heat source temperature, constant in cross-flow configuration

# Geometry of the pasta cooker:

h = 0.20 # m
w = 0.27 # m
l = 0.47 # m
V = h*w*l

# Assumptions during modeling: 
# - 0D problem: no temperature gradient inside h2o and pasta
# - constant properties for water and pasta ---> therefore, thermal resistances and cps of pasta and water are constant

# Heat pump design:
# subcritical heat pump process
# techical constraints: superheating, subcooling, optimal COP, pinch points are respected...

TASK 1: simulate pasta cooking step

In [6]:
# REQUIRED THERMAL ENERGY INPUT TO COOK THE PASTA

#Q = m_pasta*cp_pasta*(T_pasta_fin - T_pasta_in)
#print('The required thermal energy input to cook pasta is: ', Q, ' kJ')
#Q_dot = Q/t_cook
#print('The required thermal power input to cook pasta is: ', Q_dot, ' kW')

In [18]:
def h2o_energy_balance(x):
    T_in, T_fin = x
    rho = 1/FCP.state(['T','p'],[T_in,1],fluid='water',Eh='CBar')['v']
    T_water = np.linspace(T_in, T_fin, t_cook)
    T_pasta = np.linspace(T_pasta_in, T_pasta_fin, t_cook)
    Q = 0
    for t in range(0,t_cook):
        Q = Q + (T_water[t]-T_source)/R_h2o_env + (T_water[t]-T_pasta[t])/R_h2o_pasta
    return np.abs(rho*cp_h2o*V*(T_water[0] - T_water[-1]) - Q)

def h2o_constraint(x):
    T_in, T_fin = x
    return T_in - T_fin

In [20]:
NC1 = NonlinearConstraint(h2o_constraint, 0, np.inf)
bounds = [(85,93),(85,93)]
x0 = [91,87]
res = minimize(h2o_energy_balance, x0, method='SLSQP', bounds=bounds, constraints=[NC1])
print(res.x)
print(res.fun)

[93. 85.]
1352.4656648843854
